In [1]:
# ensure packages when running in a notebook are always reloaded
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

In [2]:
from typing import List
import matplotlib.pyplot as plt
import seaborn as sns
from transformers import AutoTokenizer
import matplotlib.colors as mcolors
import numpy as np

/home/codespace/.python/current/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
import sys
sys.path.append("..")

from viz.plot_token_rewards import plot_token_rewards

In [4]:
tokenizer = AutoTokenizer.from_pretrained('reshinthadith/codegen_350M_list_manip_5_len')

In [101]:
# def compare_reward_funcs(reward_funcs: List):
#     input_outputs = [
#         ["Input: 1 Output: [-4,-5,-2] Function:", "div_n(reverse([-2, -5, -4]),1)"],
#         ["Input: 1 Output: [-4,-5,-2] Function:", "div_n(reverse([-2, -5, -a]),1)"],
#         ["Input: 1 Output: [-4,-5,-2] Function:", "div_n(reverse([-2, -5, -3]),1)"],
#     ]
#     samples = [f"{input_output[0]} {input_output[1]}" for input_output in input_outputs]
#     for reward_func in reward_funcs:
#         rewards_per_sample = reward_func(samples)
#         # for sample, reward in zip(samples, rewards_per_sample):
#         for sample, input, output, reward in zip(samples, [input_output[0] for input_output in input_outputs], [input_output[1] for input_output in input_outputs], rewards_per_sample):
#             # print(f"{reward_func.__name__}: ({sample}) = {reward}")
#             # title = f"{reward_func.__name__}:\n{input} = {reward}"
#             # plotly title with newline
#             title = f"{reward_func.__name__} | {reward}<br>Prompt: {input}<br>Correct: {output}"
#             print(title)

#             tokens = tokenizer.tokenize(output)
#             plot_token_rewards(tokens, reward, title=title)

#         # plot_token_rewards(samples, rewards)

In [5]:
def compare_reward_funcs(reward_funcs: List):
    input_outputs = [
        ["Input: 1 Output: [-4,-5,-2] Function:", "div_n(reverse([-2, -5, -4]),1)", "div_n(reverse([-2, -5, -4]),1)"],
        ["Input: 1 Output: [-4,-5,-2] Function:", "div_n(reverse([-2, -5]),1)", "div_n(reverse([-2, -5, -4]),1)"],
        ["Input: 1 Output: [-4,-5,-2] Function:", "div(reverse([-2, -5, -4]),1)", "div_n(reverse([-2, -5, -4]),1)"],
        ["Input: 1 Output: [-4,-5,-2] Function:", "div_n(reverse([-2, -5, -a]),1)", "div_n(reverse([-2, -5, -4]),1)"],
        ["Input: 1 Output: [-4,-5,-2] Function:", "div_n(reverse([-2, -5, -3]),1)", "div_n(reverse([-2, -5, -4]),1)"],
    ]
    samples = [f"{input_output[0]} {input_output[1]}" for input_output in input_outputs]
    
    for sample, input, output, expected in zip(samples, [input_output[0] for input_output in input_outputs], [input_output[1] for input_output in input_outputs], [input_output[2] for input_output in input_outputs]):
        for reward_func in reward_funcs:
            kwargs = {
                'expected': [expected],
            }
            rewards_per_sample = reward_func([sample], [input], [output], tokenizer, **kwargs)
            for reward in rewards_per_sample:
                # title = f"{reward_func.__name__} | {reward}<br>Prompt: {input}<br>Correct: {output}"
                title = f"{reward_func.__name__}<br>Prompt: {input}<br>Correct: {output}"
                # print(title)

                tokens = tokenizer.tokenize(output)
                plot_token_rewards(tokens, reward, title=title)


In [28]:
from rewards import reward_basic, make_reward_token_cost, make_reward_output_length_diff, reward_func_usage, make_reward_substring_matches, reward_func_usage_dense

# reward_basic(["Input: 1 Output: [-4,-5,-2] Function: div_n(reverse([-2, -5, -4]),1)"])

compare_reward_funcs([
    # reward_basic,
    # make_reward_token_cost(20, 0.5, 0.01),
    # make_reward_token_cost(10, 0.5, 0.01),
    # make_reward_output_length_diff(10),
    # reward_func_usage,
    # make_reward_substring_matches(["div_n(", "reverse("], 0.5),
    # make_reward_substring_matches(["["], 0.5),
    # make_reward_substring_matches(["[-2, -5, -4]"], 0.5),
    reward_func_usage_dense,
])

In [51]:
text = "Your input text goes here. The quick brown fox jumps over the lazy dog!"
tokens = tokenizer.tokenize(text)
input_ids = tokenizer.convert_tokens_to_ids(tokens)
clean_tokens = [f"'{token.replace('Ġ', ' ')}'" for token in tokens]

# let's assume rewards is a list of floats, where each float represents the reward of each token
# rewards = [0.2, -0.1, 0.5, -0.2, 0.1, 0.3, 0.0, 0.4]  # Modify according to your rewards
rewards = [0] * len(tokens)
# set a random value between -1 and 1 for each token
rewards = np.random.uniform(-1, 1, len(tokens))

plot_token_rewards(clean_tokens, rewards)